In [1]:
import os

In [2]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\Detailed_Project\\Detailed_MLFLOW\\research'

In [3]:
os.chdir('../')

In [24]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\Detailed_Project\\Detailed_MLFLOW'

In [25]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    status_file: str
    unzip_data_dir: Path
    all_schema: dict

In [26]:
from src.Detailed_MLFLow_project.constants import *
from src.Detailed_MLFLow_project.utils.common import read_yaml,create_directories

class Configuration_Manager:

    def __init__(self,config_f=CONFIG_FILE_PATH,schema_f=SCHEMA_FILE_PATH,params_f=PARAMS_FILE_PATH):
        self.config=read_yaml(config_f)
        self.schema=read_yaml(schema_f)
        self.params=read_yaml(params_f)

        create_directories([self.config.artifacts_root])

    def data_validation_config(self) ->DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS

        create_directories([config.root_dir])

        get_data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.status_file,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return get_data_validation_config


In [29]:
from src import logger
import pandas as pd
class Data_validation:
    def __init__(self,config:DataValidationConfig):
        self.config=config

    def check_data_validation(self) ->bool:
        validation_status=None
        df=pd.read_csv("artifacts/data_ingestion/data/part_names.csv")
        ref_columns=list(df.columns)
        all_schema=self.config.all_schema.keys()

        try:
            for i in ref_columns:
                if i not in all_schema:
                    validation_status=False
                    logger.info(f"feature {i} does not exist")
                    with open(self.config.status_file, "w") as stat:
                        stat.write(f"Validation Status: {validation_status}")
                
                else:
                    validation_status=True
                    logger.info(f"feature {i} matched")
                    with open(self.config.status_file, "w") as stat:
                        stat.write(f"Validation Status: {validation_status}")


        except Exception as e:
            raise e

In [30]:
try:
    con1=Configuration_Manager()
    data_val_configure=con1.data_validation_config()
    data_val=Data_validation(config=data_val_configure)
    data_val.check_data_validation()
except Exception as e:
    raise e

[2024-03-21 14:59:37,559: INFO: common: yaml file {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/hrishikesh147/MLProject/raw/main/part_namesz.zip', 'local_data_file': 'artifacts/data_ingestion/zipped_data.zip', 'unzip_data': 'artifacts/data_ingestion/data'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'status_file': 'artifacts/data_validation/status.txt', 'unzip_data_dir': 'artifacts/data_ingestion/data/part_names.csv'}} loaded successfully]
[2024-03-21 14:59:37,568: INFO: common: yaml file {'COLUMNS': {'VehicleModel': 'object', 'VehicleYear': 'int64', 'PartName': 'object', 'HoursOfOperation': 'float64', 'SupplierName': 'object', 'ClaimDate': 'object', 'SettlementAmount': 'float64', 'MaintenanceFrequency': 'object', 'EnvironmentCondition': 'object', 'OperationalIntensity': 'object', 'WarrantyStatus': 'object', 'PreviousFailures': 'int64', 'pass_fail': 'object'}, 'TARGET_COLUMN': {'name': 'pas